In [10]:
import json
import hashlib
import os
import random
import requests
import time

from datetime import datetime

In [11]:
# Declarando as variaveis
DATETIME_PATTERN = '%Y-%m-%d %H:%M:%S'
PAYLOAD_PATH = "logs/received/instructions/"
DIR_MARVEL_GET = "logs/get_marvel"
DIR_MARVEL_GET_ERROR = "logs/error_get_marvel"
POST_PROFESSOR = "logs/post_professor"
URL_MARVEL_BASE = "http://gateway.marvel.com/v1/public"
POST_PROFESSOR_URL = ""
os.makedirs(DIR_MARVEL_GET, exist_ok=True)
os.makedirs(DIR_MARVEL_GET_ERROR, exist_ok=True)
os.makedirs(POST_PROFESSOR, exist_ok=True)

In [12]:
# Escrever arquivos
def write_file(data_, path, id):
    with open(f"{path}/{id}.json", "w") as file:
        file.write(json.dumps(data_))

In [13]:
# Ler os arquivos
def read_files(path):
    payload_list = []

    file_list = os.listdir(path)

    for file_ in file_list:
        with open(f"{path}/{file_}") as file:
            payload_list.append(json.load(file))
    return payload_list

In [14]:
# Criar o hash md5 para a chamada de api da marvel
def create_hash(timestamp, private_key, public_key):

    str_to_hash = str(timestamp) + str(private_key) + str(public_key)
    hash_md5 = hashlib.md5(str_to_hash.encode())
    return hash_md5.hexdigest()

In [15]:
# Ler o arquivo de credenciais da marvel
with open("api.json") as file:
    credentials = json.load(file)

public_key_ = credentials['marvel']['public']
private_key_ = credentials['marvel']['private']


In [23]:
# Carrega uma lista contendo todas as payloads em conformidade recebidas pelo server
payload_list = read_files(PAYLOAD_PATH)

# Realiza um loop para retornar todos os dados brutos da API da Marvel, para cada requisição encontrada na lista
for payload in payload_list:
    request_url = f"{URL_MARVEL_BASE}/{payload['body']['entity_name']}/{payload['body']['entity_id']}"
    ts_ = datetime.now().timestamp()

    params_ = {
        "ts": ts_,
        "apikey": public_key_,
        "hash": create_hash(timestamp=ts_, private_key=private_key_, public_key=public_key_)
    }

    response = requests.get(url=request_url, params=params_)
    response_dict = response.json()
    id_ = f"{payload['body']['entity_name']}_{payload['body']['instruction_id']}_{payload['body']['entity_id']}_{datetime.now().strftime('%Y%m%d-%H%M%S-%f')}"

    if response.status_code == 200:
        write_file(data_=response_dict, path=DIR_MARVEL_GET, id=id_)
    else:
        print(f"SOMETHING WENT WRONG HERE: {payload}")
        data_dict_ = {
            "instruction_id": payload['body']['instruction_id'],
            "entity_name": payload['body']['entity_name'],
            "entity_id": payload['body']['entity_id'],
            "error": response_dict
        }
        write_file(data_=data_dict_, path=DIR_MARVEL_GET_ERROR, id=id_)

    time.sleep(0.5)

In [26]:
# Para cada payload recuperada, realiza a formatação da payload de resposta, de acordo com a entidade
file_name_list = os.listdir(DIR_MARVEL_GET)

for file_name in file_name_list:
    with open(f"{DIR_MARVEL_GET}/{file_name}") as file:
        raw_data = json.load(file)
    
    file_name_parts = file_name.split("_")  # 0=entity_name, 1=instruction_id, 2=entity_id, 3=execution_id
    body = {}
    results = raw_data['data']['results'][0]

    if file_name_parts[0] == "characters":
        body = {
            "character_name": "",
            "comics_available": 0,
            "url_wiki": ""
        }

        body['character_name'] = results['name']
        body['comics_available'] = results['comics']['available']
        body['url_wiki'] = next((x for x in results['urls'] if x['type'] == 'wiki'), None)['url']

    elif file_name_parts[0] == "comics":
        body = {
            "title": "",
            "pages": 0,
            "url_detail": ""
        }
        body['title'] = results['title']
        body['pages'] = results['pageCount']
        body['url_detail'] = next((x for x in results['urls'] if x['type'] == 'detail'), None)['url']

    elif file_name_parts[0] == "creators":
        body = {
            "creator_full_name": "",
            "comics": 0,
            "series": 0
        }
        body['creator_full_name'] = results['fullName']
        body['comics'] = results['comics']['available']
        body['series'] = results['series']['available']

    elif file_name_parts[0] == "events":
        body = {
            "title": "",
            "start_date": "",
            "creators": 0
        }
        body['title'] = results['title']
        body['start_date'] = results['start']
        body['creators'] = results['creators']['available']

    elif file_name_parts[0] == "series":
        body = {
            "title": "",
            "start_date": 0,
            "stories": 0
        }
        body['title'] = results['title']
        body['start_date'] = results['startYear']
        body['stories'] = results['stories']['available']

    elif file_name_parts[0] == "stories":
        body = {
            "title": "",
            "originalIssue_name": "",
            "originalIssue_resource": ""
        }
        body['title'] = results['title']
        body['originalIssue_name'] = results['originalIssue']['name']
        body['originalIssue_resource'] = results['originalIssue']['resourceURI']
    
    # Ao fim, formatada e grava o arquivo em um diretório para realização do POST de resposta
    post_dict = {
        "type": "answers",
        "body": {
            "instruction_id": int(file_name_parts[1]),
            "nome_estudante": "Iago Freitas",
            "timestamp": datetime.now().strftime(DATETIME_PATTERN),
            "mensagem": "Resposta do exercicio",
            "resultado": body
        }
    }
    execution_id_ = f"{file_name_parts[0]}_{file_name_parts[1]}_{file_name_parts[2]}_{datetime.now().strftime('%Y%m%d-%H%M%S-%f')}"
    write_file(data_=post_dict, path=POST_PROFESSOR, id=execution_id_)

In [ ]:
# Carrega uma lista contendo todos os payloads de resposta
payload_list = read_files(POST_PROFESSOR)

# Realiza um loop para realizar todos os POST de resposta ao cliente
for payload in payload_list:
    response = requests.post(POST_PROFESSOR_URL, json=payload)

    print(response)
    print(response.content)